In [ ]:
#load the necessary modules
from imutils import paths
from random import shuffle
import os
import numpy as np
import skimage
from skimage.measure import compare_ssim
import cv2
from shutil import copyfile
import random
#code adapted from https://ourcodeworld.com/articles/read/991/how-to-calculate-the-structural-similarity-index-ssim-between-two-images-with-python

#### Pictures of each of the individuals should be in different folders. For example if training a model to classify 3 classes the directory of the original dataset should be:
    PATH/TO/ORIGINAL_DATASET/
                         Class_A/
                                 Class_A_image1.jpg
                                 Class_A_image2.jpg
                                 Class_A_image3.jpg
                                 .
                                 .
                                 .
                                 
                         Class_B/
                                 Class_B_image1.jpg
                                 .
                                 .
                                 .
                         Class_C/
                                 Class_C_image1.jpg

In [ ]:
#Create a list with the name of the birds to loop through all individuals
birds=["007",
       "103",
       "107",
       "179",
       "194"]

In [ ]:
#Total number of pictures to be moved
number_of_pictures=160
#similarity threshold to decide if the picture should be included
threshold=0.50
#Create a variable to note if the picture should be selected (if it is not similar to any other picture)
move=0
#get the path to the dataset
Dataset="/PATH/TO/ORIGINAL_DATASET"
#loop through all birds identities and copy the pictures to the new folder if the similarity index 
#is lower than the threshold

for bird in birds:
    #create an empty list to store the pictures already copied for the current bird
    imagePaths_new_folder=[]
    #loop to copy the pictures to the new folder for the current bird
    #until enough pictures are copied
    while len(imagePaths_new_folder)<number_of_pictures:  
        #list all pictures in the original dataset
        imagePaths = sorted(list(paths.list_images(Dataset)))
        #select pictures for the current bird
        imagePaths =  [pic for pic in imagePaths if bird in pic]
        #get the path to the folder where the pictures will be copied
        destination_folder="/PATH/TO/THE_NEW_DATASET/"+bird
        #list all pictures that are already in the destination_folder
        imagePaths_new_folder = sorted(list(paths.list_images(destination_folder)))

        #if the destination folder is empty (first picture) copy a random picture to start
        if len(imagePaths_new_folder)==0:
            #select a picture
            pic_dir=imagePaths[random.randint(0, len(imagePaths)-1)]
            #copy the picture to the new folder
            copyfile(pic_dir, destination_folder+"/"+pic_dir.split("/")[-1:][0])
        
        #if there are already pictures in the destination folder
        #compare a random picture with all pictures already copied
        #using the similarity index
        if len(imagePaths_new_folder)>0:
            #seletct a picture
            pic_dir=imagePaths[random.randint(0,len(imagePaths)-1)]
            #open and resize the picture (it can be to the size used by the CNN) so that all pictures have the same size
            picture = cv2.imread(pic_dir)
            picture=cv2.resize(picture, (224,224))
            #compare the picture to all other pictures already in the folder
            for c in range(0, len(imagePaths_new_folder)):
                tmp_pic=cv2.imread(imagePaths_new_folder[c])
                tmp_pic=cv2.resize(tmp_pic, (224,224))
                grayA = cv2.cvtColor(picture, cv2.COLOR_BGR2GRAY)
                grayB = cv2.cvtColor(tmp_pic, cv2.COLOR_BGR2GRAY)
                #compute the similarity index
                (score, diff) = compare_ssim(grayA, grayB, full=True)
                diff = (diff * 255).astype("uint8")
                #if the similarity index score is larger than the threshold
                #mark the picture as not fit to be included in the new dataset
                #and break the loop
                if score>threshold:
                    move=1
                    break
            #if the randomly selected picture was not similar to any of the
            #pictures already in the new dataset copy it
            if move==0:
                copyfile(pic_dir, destination_folder+"/"+pic_dir.split("/")[-1:][0])
            move=0
            